<a href="https://colab.research.google.com/github/wenchun0731/city-hunter/blob/main/%E9%97%96%E7%B4%85%E7%87%88%E6%95%B4%E5%90%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#掛載雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#下載ultralytics開源庫
!pip install ultralytics

#下載YOLOv8並檢查模型配置
from ultralytics import YOLO
import os
from IPython.display import display,Image
from IPython import display
display.clear_output()
!yolo model=checks

!pip install deep-sort-realtime
!pip install easyocr
!pip install mysql-connector-python
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install pytesseract
!pip install pillow

WARNING ⚠️ 'mode' argument is missing. Valid modes are {'track', 'predict', 'train', 'val', 'benchmark', 'export'}. Using default 'mode=train'.
Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 756, in entrypoint
    stem = Path(model).stem.lower()
  File "/usr/lib/python3.10/pathlib.py", line 960, in __new__
    self = cls._from_parts(args)
  File "/usr/lib/python3.10/pathlib.py", line 594, in _from_parts
    drv, root, parts = self._parse_args(args)
  File "/usr/lib/python3.10/pathlib.py", line 578, in _parse_args
    a = os.fspath(a)
TypeError: expected str, bytes or os.PathLike object, not module
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━

In [ ]:
#Deep sort 原程式
#DeepSort(使用自己的訓練檔)
!pip install ultralytics
!pip install deep-sort-realtime
import numpy as np
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab.patches import cv2_imshow

#輸入
model = YOLO('/content/drive/MyDrive/Finetune/Licence_Finetune/train/train/weights/best.pt')
cap = cv2.VideoCapture('/content/drive/MyDrive/Finetune/Licence_Finetune/test_video/test_video4.mp4')  #輸入影片
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fNUMS = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
videoWriter = cv2.VideoWriter("/content/drive/MyDrive/test-video0905.mp4", fourcc, fps, size)#輸出位置


#nms_max_overlap非極大值抑制（NMS）過程中允許的最大重疊比例
#nn_budget控制最近鄰搜尋中使用的樣本數量的上限
#max_cosine_distance 控制追蹤器在匹配檢測框與追蹤框時，外觀特徵向量之間的最大允許餘弦距離，一般設置為0到1之間
tracker = DeepSort(
    max_age=60,  # 增加此值以保持更長時間的追蹤
    max_cosine_distance=0.4,  # 減少此值以提高特徵匹配的精確度一般設置為0到1之間
    nn_budget=100  # 調整最近鄰搜尋的樣本數量
)

# 定義顏色字典
colors = {
    'ambulance':(0,255,0),   # 綠色
    'bus': (0,130,255),     # 青色
    'car': (0,0,255),      # 藍色
    'fire truck': (255,0,0),  # 紅色
    'licence': (60,0,128),   # 紫色
    'person': (255,255,0),   # 黃色
    'police': (0,0,0),     # 黑色
    'traffic light': (255,0,255) # 粉紅色
}

def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
    cv2.rectangle(image, p1, p2, color, thickness=3, lineType=cv2.LINE_AA)
    if label:
        # 在這裡將座標加入標籤內容
        label_with_coords = f"{label} ({p1[0]},{p1[1]})-({p2[0]},{p2[1]})"
        w, h = cv2.getTextSize(label_with_coords, 0, fontScale=2 / 3, thickness=3)[0]
        outside = p1[1] - h >= 3
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)
        cv2.putText(image,
                    label_with_coords, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                    0, 2/3, txt_color, thickness=3, lineType=cv2.LINE_AA)


while cap.isOpened():
    success, frame = cap.read()

    if success:
        results = model(frame, conf=0.5)#置信值
        outputs = results[0].boxes.data.cpu().numpy()

        detections = []

        if outputs is not None:
            for output in outputs:
                x1, y1, x2, y2 = list(map(int, output[:4]))
                class_id = int(output[5])

                # 根據模型的類別對應，將class_id對應到具體的類別名稱
                if class_id == 0:
                    label = 'ambulance'
                elif class_id == 1:
                    label = 'bus'
                elif class_id == 2:
                    label = 'car'
                elif class_id == 3:
                    label = 'fire truck'
                elif class_id == 4:
                    label = 'licence'
                elif class_id == 5:
                    label = 'person'
                elif class_id == 6:
                    label = 'police'
                elif class_id == 7:
                    label = 'traffic light'

                color = colors.get(label, (128, 128, 128))  # 默認顏色

                detections.append(([x1, y1, int(x2-x1), int(y2-y1)], output[4], label))

            tracks = tracker.update_tracks(detections, frame=frame)

            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                bbox = track.to_ltrb()

                # 獲取對應的顏色
                label = track.det_class
                color = colors.get(label, (128, 128, 128))  # 默認顏色
                box_label(frame, bbox, '#' + str(int(track_id)) + label, color)

        cv2.putText(frame, "https://blog.csdn.net/zhaocj", (25, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        videoWriter.write(frame)

    else:
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()


0: 384x640 2 cars, 40.5ms
Speed: 1.8ms preprocess, 40.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 30.3ms
Speed: 3.8ms preprocess, 30.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 30.3ms
Speed: 2.3ms preprocess, 30.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 30.4ms
Speed: 2.1ms preprocess, 30.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 25.7ms
Speed: 2.0ms preprocess, 25.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 47.3ms
Speed: 2.5ms preprocess, 47.3ms inference, 5.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 25.4ms
Speed: 2.0ms preprocess, 25.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 25.3ms
Speed: 2.1ms preprocess, 25.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x6

In [ ]:
# 標停止線
!pip install ultralytics
!pip install deep-sort-realtime

import numpy as np
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab.patches import cv2_imshow

# 載入模型
model = YOLO('/content/drive/MyDrive/Finetune/Licence_Finetune/train/train/weights/best.pt')
cap = cv2.VideoCapture('/content/drive/MyDrive/Finetune/Licence_Finetune/test_video/IMG_0666.MOV')  # 輸入影片
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
videoWriter = cv2.VideoWriter("/content/drive/MyDrive/test-video0930.mp4", fourcc, fps, size)  # 輸出位置

# 初始化 DeepSort 追蹤器
tracker = DeepSort(
    max_age=60,
    max_cosine_distance=0.4,
    nn_budget=100
)

# 定義顏色字典
colors = {
    'ambulance': (0, 255, 0),
    'bus': (0, 130, 255),
    'car': (0, 0, 255),
    'fire truck': (255, 0, 0),
    'licence': (60, 0, 128),
    'person': (255, 255, 0),
    'police': (0, 0, 0),
    'traffic light': (255, 0, 255)
}

# 偵測紅綠燈顏色
def get_traffic_light_color(detections):
    for det in detections:
        label = det[2]
        if label == 'traffic light':
            return label  # 假設模型可以辨識紅燈
    return None

# 判斷闖紅燈
def check_red_light_violation(tracks, traffic_light_color, frame):
    if traffic_light_color != 'red':
        return

    # 固定的停止線座標：x從630到900，y為330
    stop_line_y = 330
    for track in tracks:
        if track.det_class in ['car', 'bus', 'ambulance', 'fire truck']:  # 只考慮車輛類別
            bbox = track.to_ltrb()
            if bbox[1] > stop_line_y:  # 車輛超過停止線
                print(f"Car {track.track_id} ran a red light!")
                box_label(frame, bbox, f'Violation #{track.track_id}', (0, 0, 255))

# 畫框標籤
def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
    cv2.rectangle(image, p1, p2, color, thickness=3, lineType=cv2.LINE_AA)
    if label:
        label_with_coords = f"{label} ({p1[0]},{p1[1]})-({p2[0]},{p2[1]})"
        w, h = cv2.getTextSize(label_with_coords, 0, fontScale=2 / 3, thickness=3)[0]
        outside = p1[1] - h >= 3
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)
        cv2.putText(image, label_with_coords, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                    0, 2 / 3, txt_color, thickness=3, lineType=cv2.LINE_AA)

# 主程式迴圈
while cap.isOpened():
    success, frame = cap.read()

    if success:
        results = model(frame, conf=0.5)  # 偵測物體
        outputs = results[0].boxes.data.cpu().numpy()

        detections = []

        if outputs is not None:
            for output in outputs:
                x1, y1, x2, y2 = list(map(int, output[:4]))
                class_id = int(output[5])

                label = ['ambulance', 'bus', 'car', 'fire truck', 'licence', 'person', 'police', 'traffic light'][class_id]
                color = colors.get(label, (128, 128, 128))
                detections.append(([x1, y1, int(x2 - x1), int(y2 - y1)], output[4], label))

            traffic_light_color = get_traffic_light_color(detections)  # 偵測紅綠燈顏色
            tracks = tracker.update_tracks(detections, frame=frame)  # 更新追蹤資訊

            check_red_light_violation(tracks, traffic_light_color, frame)  # 檢查是否闖紅燈

            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                bbox = track.to_ltrb()

                label = track.det_class
                color = colors.get(label, (128, 128, 128))
                box_label(frame, bbox, '#' + str(int(track_id)) + label, color)

        # 在畫面中繪製固定的黃色停止線）
        cv2.line(frame, (800, 820), (1500, 780), (0, 255, 255), 5)  # 黃色停止線

        cv2.putText(frame, "https://blog.csdn.net/zhaocj", (25, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        videoWriter.write(frame)

    else:
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()


串流輸出內容已截斷至最後 5000 行。
0: 384x640 (no detections), 22.9ms
Speed: 3.2ms preprocess, 22.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.9ms
Speed: 4.1ms preprocess, 22.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.9ms
Speed: 3.2ms preprocess, 22.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.8ms
Speed: 2.9ms preprocess, 22.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.9ms
Speed: 3.1ms preprocess, 22.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.9ms
Speed: 2.9ms preprocess, 22.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.9ms
Speed: 3.3ms preprocess, 22.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.9ms
Speed: 2.9ms 

In [ ]:
# 紀錄經過停止線的車
!pip install ultralytics
!pip install deep-sort-realtime

import numpy as np
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# 載入模型
model = YOLO('/content/drive/MyDrive/Finetune/Licence_Finetune/train/train/weights/best.pt')
cap = cv2.VideoCapture('/content/drive/MyDrive/Finetune/Licence_Finetune/test_video/IMG_0666.MOV')  # 輸入影片
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
videoWriter = cv2.VideoWriter("/content/drive/MyDrive/test-video0930.mp4", fourcc, fps, size)  # 輸出位置

# 初始化 DeepSort 追蹤器
tracker = DeepSort(
    max_age=60,
    max_cosine_distance=0.5,
    nn_budget=100
)

# 定義顏色字典
colors = {
    'ambulance': (0, 255, 0),
    'bus': (0, 130, 255),
    'car': (0, 0, 255),
    'fire truck': (255, 0, 0),
    'licence': (60, 0, 128),
    'person': (255, 255, 0),
    'police': (0, 0, 0),
    'traffic light': (255, 0, 255)
}

# 用來存儲通過停止線的車輛資訊的列表
vehicles_passed_stop_line = []
recorded_vehicles = set()  # 記錄已經通過停止線的車輛ID

# 畫框標籤
def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))

    # 畫出矩形框
    cv2.rectangle(image, p1, p2, color, thickness=3, lineType=cv2.LINE_AA)
    if label:
        w, h = cv2.getTextSize(label, 0, fontScale=2 / 3, thickness=3)[0]
        outside = p1[1] - h >= 3
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)
        cv2.putText(image,
                    label, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                    0, 2/3, txt_color, thickness=3, lineType=cv2.LINE_AA)

# 判斷車輛是否超過停止線
def check_vehicle_passing_stop_line(tracks, frame_num, frame):
    # 固定的停止線座標：x從630到900，y為430
    stop_line_y = 430
    for track in tracks:
        if track.det_class in ['car', 'bus', 'ambulance', 'fire truck']:  # 只考慮車輛類別
            bbox = track.to_ltrb()
            if bbox[1] > stop_line_y and track.track_id not in recorded_vehicles:  # 車輛超過停止線且未被記錄
                # 計算時間戳（影片當前幀數除以FPS）
                timestamp = frame_num / fps
                # 保存車輛ID、座標、時間戳
                vehicles_passed_stop_line.append({
                    'track_id': track.track_id,
                    'coordinates': bbox,
                    'timestamp': timestamp
                })
                # 將車輛ID加入已記錄集合
                recorded_vehicles.add(track.track_id)
                box_label(frame, bbox, f'Passed #{track.track_id}', (0, 255, 255))

# 主程式迴圈
frame_num = 0
while cap.isOpened():
    success, frame = cap.read()

    if success:
        frame_num += 1
        results = model(frame, conf=0.4, iou=0.8)  # 偵測物體
        outputs = results[0].boxes.data.cpu().numpy()

        detections = []

        if outputs is not None:
            for output in outputs:
                x1, y1, x2, y2 = list(map(int, output[:4]))
                class_id = int(output[5])

                label = ['ambulance', 'bus', 'car', 'fire truck', 'licence', 'person', 'police', 'traffic light'][class_id]
                color = colors.get(label, (128, 128, 128))
                detections.append(([x1, y1, int(x2 - x1), int(y2 - y1)], output[4], label))

            tracks = tracker.update_tracks(detections, frame=frame)  # 更新追蹤資訊

            check_vehicle_passing_stop_line(tracks, frame_num, frame)  # 檢查車輛是否通過停止線

            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                bbox = track.to_ltrb()

                label = track.det_class
                color = colors.get(label, (128, 128, 128))
                box_label(frame, bbox, '#' + str(int(track_id)) + label, color)

        # 在畫面中繪製固定的黃色停止線（x: 630-1200，y: 430）
        cv2.line(frame, (800, 820), (1500, 780), (0, 255, 255), 5)  # 黃色停止線

        cv2.putText(frame, "https://blog.csdn.net/zhaocj", (25, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        videoWriter.write(frame)

    else:
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()

# 打印所有通過停止線的車輛資訊
print("Vehicles that passed the stop line:")
for vehicle in vehicles_passed_stop_line:
    print(f"Vehicle ID: {vehicle['track_id']}, Coordinates: {vehicle['coordinates']}, Time: {vehicle['timestamp']:.2f} seconds")


串流輸出內容已截斷至最後 5000 行。
0: 384x640 (no detections), 24.7ms
Speed: 3.2ms preprocess, 24.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 23.2ms
Speed: 3.8ms preprocess, 23.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 22.8ms
Speed: 3.6ms preprocess, 22.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 22.8ms
Speed: 3.9ms preprocess, 22.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 22.7ms
Speed: 3.4ms preprocess, 22.7ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 25.2ms
Speed: 5.3ms preprocess, 25.2ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 22.8ms
Speed: 3.2ms preprocess, 22.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.7ms
Speed: 3.0ms preprocess, 22.7ms inference, 0.8ms postprocess pe

In [ ]:
#測試紀錄車牌
!pip install ultralytics
!pip install deep-sort-realtime
!pip install easyocr

import numpy as np
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import easyocr

# 初始化 EasyOCR
reader = easyocr.Reader(['en'])

# 載入模型
model = YOLO('/content/drive/MyDrive/Finetune/Licence_Finetune/train/train/weights/best.pt')
cap = cv2.VideoCapture('/content/drive/MyDrive/Finetune/Licence_Finetune/test_video/IMG_0666.MOV')  # 輸入影片
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
videoWriter = cv2.VideoWriter("/content/drive/MyDrive/test-video0930-test.mp4", fourcc, fps, size)  # 輸出位置

# 初始化 DeepSort 追蹤器
tracker = DeepSort(
    max_age=60,
    max_cosine_distance=0.5,
    nn_budget=100
)

# 定義顏色字典
colors = {
    'ambulance': (0, 255, 0),
    'bus': (0, 130, 255),
    'car': (0, 0, 255),
    'fire truck': (255, 0, 0),
    'licence': (60, 0, 128),
    'person': (255, 255, 0),
    'police': (0, 0, 0),
    'traffic light': (255, 0, 255)
}

# 用來存儲通過停止線的車輛資訊的列表
vehicles_passed_stop_line = []
recorded_vehicles = set()  # 記錄已經通過停止線的車輛ID

# 畫框標籤
def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))

    # 畫出矩形框
    cv2.rectangle(image, p1, p2, color, thickness=3, lineType=cv2.LINE_AA)
    if label:
        w, h = cv2.getTextSize(label, 0, fontScale=2 / 3, thickness=3)[0]
        outside = p1[1] - h >= 3
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)
        cv2.putText(image,
                    label, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                    0, 2/3, txt_color, thickness=3, lineType=cv2.LINE_AA)

# 車牌文字提取函數
def extract_license_plate_text(frame, bbox):
    # 截取車牌部分
    x1, y1, x2, y2 = list(map(int, bbox))
    license_plate_image = frame[y1:y2, x1:x2]

    # 使用 EasyOCR 來讀取車牌號碼
    result = reader.readtext(license_plate_image)
    if result:
        return result[0][-2]  # 提取最有可能的文字
    return ''

# 偵測車牌的過程
def detect_license_plates(outputs, frame):
    license_plates = []
    for output in outputs:
        class_id = int(output[5])
        if class_id == 4:  # '4' 是車牌的類別（假設）
            bbox = output[:4]
            license_plate_text = extract_license_plate_text(frame, bbox)  # 提取車牌文字
            license_plates.append({
                'bbox': bbox,
                'text': license_plate_text
            })
    return license_plates

# 判斷車輛是否超過停止線
def check_vehicle_passing_stop_line(tracks, frame_num, frame, license_plates):
    # 固定的停止線座標：x從630到900，y為430
    stop_line_y = 430
    for track in tracks:
        if track.det_class in ['car', 'bus', 'ambulance', 'fire truck']:  # 只考慮車輛類別
            bbox = track.to_ltrb()

            # 檢查車輛是否超過停止線且未被記錄
            if bbox[1] > stop_line_y and track.track_id not in recorded_vehicles:
                timestamp = frame_num / fps

                # 找到與車輛位置重疊的車牌
                license_plate_text = ''
                for license_plate in license_plates:
                    lp_bbox = license_plate['bbox']
                    if (lp_bbox[0] >= bbox[0] and lp_bbox[1] >= bbox[1] and
                        lp_bbox[2] <= bbox[2] and lp_bbox[3] <= bbox[3]):
                        license_plate_text = license_plate['text']  # 提取車牌號碼
                        break

                # 保存車輛ID、座標、車牌號碼、時間戳
                vehicles_passed_stop_line.append({
                    'track_id': track.track_id,
                    'coordinates': bbox,
                    'license_plate': license_plate_text,
                    'timestamp': timestamp
                })
                recorded_vehicles.add(track.track_id)
                box_label(frame, bbox, f'Passed #{track.track_id} {license_plate_text}', (0, 255, 255))

# 主程式迴圈
frame_num = 0
while cap.isOpened():
    success, frame = cap.read()

    if success:
        frame_num += 1
        results = model(frame, conf=0.4, iou=0.8)  # 偵測物體
        outputs = results[0].boxes.data.cpu().numpy()

        detections = []

        if outputs is not None:
            # 偵測車牌
            license_plates = detect_license_plates(outputs, frame)

            for output in outputs:
                x1, y1, x2, y2 = list(map(int, output[:4]))
                class_id = int(output[5])

                label = ['ambulance', 'bus', 'car', 'fire truck', 'licence', 'person', 'police', 'traffic light'][class_id]
                color = colors.get(label, (128, 128, 128))
                detections.append(([x1, y1, int(x2 - x1), int(y2 - y1)], output[4], label))

            tracks = tracker.update_tracks(detections, frame=frame)  # 更新追蹤資訊

            check_vehicle_passing_stop_line(tracks, frame_num, frame, license_plates)  # 檢查車輛是否通過停止線

            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                bbox = track.to_ltrb()

                label = track.det_class
                color = colors.get(label, (128, 128, 128))
                box_label(frame, bbox, '#' + str(int(track_id)) + label, color)

        # 在畫面中繪製固定的黃色停止線（x: 630-1200，y: 430）
        cv2.line(frame, (800, 820), (1500, 780), (0, 255, 255), 5)  # 黃色停止線

        cv2.putText(frame, "https://blog.csdn.net/zhaocj", (25, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        videoWriter.write(frame)

    else:
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()

# 打印所有通過停止線的車輛資訊
print("Vehicles that passed the stop line:")
for vehicle in vehicles_passed_stop_line:
    print(f"Vehicle ID: {vehicle['track_id']}, Coordinates: {vehicle['coordinates']}, License Plate: {vehicle['license_plate']}, Time: {vehicle['timestamp']:.2f} seconds")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 11.0 MB/s eta 0:00:00


Progress: |██████████████████████████████████████████████████| 100.0% Complete

串流輸出內容已截斷至最後 5000 行。
0: 384x640 (no detections), 2858.5ms
Speed: 4.3ms preprocess, 2858.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 1883.2ms
Speed: 4.2ms preprocess, 1883.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1927.6ms
Speed: 4.6ms preprocess, 1927.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1900.9ms
Speed: 4.7ms preprocess, 1900.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1948.3ms
Speed: 3.9ms preprocess, 1948.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 2149.7ms
Speed: 4.4ms preprocess, 2149.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 3006.2ms
Speed: 7.8ms preprocess, 3006.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2380.2ms
Speed: 7.3ms preprocess, 2380.2ms

In [ ]:
#存圖片到雲端
import numpy as np
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import easyocr
import os

# 初始化 EasyOCR
reader = easyocr.Reader(['en'])

# 載入 YOLO 模型
model = YOLO('/content/drive/MyDrive/Finetune/Licence_Finetune/train/train/weights/best.pt')

# 輸入影片
cap = cv2.VideoCapture('/content/drive/MyDrive/Finetune/Licence_Finetune/test_video/IMG_0666.MOV')
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
videoWriter = cv2.VideoWriter("/content/drive/MyDrive/test-video1011.mp4", fourcc, fps, size)

# 初始化 DeepSort 追蹤器
tracker = DeepSort(max_age=60, max_cosine_distance=0.5, nn_budget=100)

# 定義顏色字典
colors = {
    'ambulance': (0, 255, 0),
    'bus': (0, 130, 255),
    'car': (0, 0, 255),
    'fire truck': (255, 0, 0),
    'licence': (60, 0, 128),
    'person': (255, 255, 0),
    'police': (0, 0, 0),
    'traffic light': (255, 0, 255)
}

# 確保所有截圖保存到名為 'photos' 的資料夾
output_folder = '/content/drive/MyDrive/photos'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 用來存儲通過停止線的車輛資訊的列表
vehicles_passed_stop_line = []
recorded_vehicles = set()  # 記錄已經通過停止線並截圖的車輛ID

# 畫框標籤
def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))

    # 畫出矩形框
    cv2.rectangle(image, p1, p2, color, thickness=3, lineType=cv2.LINE_AA)
    if label:
        w, h = cv2.getTextSize(label, 0, fontScale=2 / 3, thickness=3)[0]
        outside = p1[1] - h >= 3
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)
        cv2.putText(image,
                    label, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                    0, 2/3, txt_color, thickness=3, lineType=cv2.LINE_AA)

# 車牌文字提取函數
def extract_license_plate_text(frame, bbox):
    # 截取車牌部分
    x1, y1, x2, y2 = list(map(int, bbox))
    license_plate_image = frame[y1:y2, x1:x2]

    # 使用 EasyOCR 來讀取車牌號碼
    result = reader.readtext(license_plate_image)
    if result:
        return result[0][-2]  # 提取最有可能的文字
    return ''

# 偵測車牌的過程
def detect_license_plates(outputs, frame):
    license_plates = []
    for output in outputs:
        class_id = int(output[5])
        if class_id == 4:  # '4' 是車牌的類別（假設）
            bbox = output[:4]
            license_plate_text = extract_license_plate_text(frame, bbox)  # 提取車牌文字
            if license_plate_text:  # 如果偵測到車牌文字，才加入列表
                license_plates.append({
                    'bbox': bbox,
                    'text': license_plate_text
                })
    return license_plates

# 保存車輛裁切圖片的函數，並存到同一個資料夾
def save_cropped_vehicle_image(frame, bbox, track_id, timestamp, license_plate_text):
    # 截取車輛圖像
    x1, y1, x2, y2 = list(map(int, bbox))
    cropped_vehicle = frame[y1:y2, x1:x2]

    # 生成文件名稱，包含秒數（後兩位）和車牌號碼
    timestamp_str = f"{int(timestamp):02d}"  # 秒數
    file_name = f"vehicle_{track_id}_time_{timestamp_str}_{license_plate_text}.jpg"
    file_path = os.path.join(output_folder, file_name)

    # 保存裁切後的圖片到同一資料夾
    cv2.imwrite(file_path, cropped_vehicle)
    print(f"Vehicle image saved: {file_path}")

# 判斷車輛是否超過停止線，並且偵測到車牌號碼
def check_vehicle_passing_stop_line(tracks, frame_num, frame, license_plates):
    # 固定的停止線座標：x從630到900，y為430
    stop_line_y = 430
    for track in tracks:
        if track.det_class in ['car', 'bus', 'ambulance', 'fire truck']:  # 只考慮車輛類別
            bbox = track.to_ltrb()

            # 檢查車輛是否超過停止線且未被記錄
            if bbox[1] > stop_line_y and track.track_id not in recorded_vehicles:
                timestamp = frame_num / fps

                # 找到與車輛位置重疊的車牌
                license_plate_text = ''
                for license_plate in license_plates:
                    lp_bbox = license_plate['bbox']
                    if (lp_bbox[0] >= bbox[0] and lp_bbox[1] >= bbox[1] and
                        lp_bbox[2] <= bbox[2] and lp_bbox[3] <= bbox[3]):
                        license_plate_text = license_plate['text']  # 提取車牌號碼
                        break

                # 只有當偵測到車牌文字時才截圖並記錄
                if license_plate_text:
                    # 保存車輛ID、座標、車牌號碼、時間戳
                    vehicles_passed_stop_line.append({
                        'track_id': track.track_id,
                        'coordinates': bbox,
                        'license_plate': license_plate_text,
                        'timestamp': timestamp
                    })

                    # 記錄該車輛ID以避免重複截圖
                    recorded_vehicles.add(track.track_id)

                    # 畫框標籤
                    box_label(frame, bbox, f'Passed #{track.track_id} {license_plate_text}', (0, 255, 255))

                    # 保存裁切的車輛圖像
                    save_cropped_vehicle_image(frame, bbox, track.track_id, timestamp, license_plate_text)

# 主程式迴圈
frame_num = 0
while cap.isOpened():
    success, frame = cap.read()

    if success:
        frame_num += 1
        results = model(frame, conf=0.4, iou=0.8)  # 偵測物體
        outputs = results[0].boxes.data.cpu().numpy()

        detections = []

        if outputs is not None:
            # 偵測車牌
            license_plates = detect_license_plates(outputs, frame)

            for output in outputs:
                x1, y1, x2, y2 = list(map(int, output[:4]))
                class_id = int(output[5])

                label = ['ambulance', 'bus', 'car', 'fire truck', 'licence', 'person', 'police', 'traffic light'][class_id]
                color = colors.get(label, (128, 128, 128))
                detections.append(([x1, y1, int(x2 - x1), int(y2 - y1)], output[4], label))

            tracks = tracker.update_tracks(detections, frame=frame)  # 更新追蹤資訊

            check_vehicle_passing_stop_line(tracks, frame_num, frame, license_plates)  # 檢查車輛是否通過停止線

            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                bbox = track.to_ltrb()

                label = track.det_class
                color = colors.get(label, (128, 128, 128))
                box_label(frame, bbox, '#' + str(int(track_id)) + label, color)

        # 繪製固定的黃色停止線（x: 800-1500，y: 780-820）
        cv2.line(frame, (800, 820), (1500, 780), (0, 255, 255), 5)  # 黃色停止線

        cv2.putText(frame, "https://blog.csdn.net/zhaocj", (25, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        videoWriter.write(frame)

        if frame_num % 30 == 0:
            print(f"Processed frame {frame_num}")
    else:
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()


串流輸出內容已截斷至最後 5000 行。
Speed: 3.2ms preprocess, 22.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 22.1ms
Speed: 3.3ms preprocess, 22.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 21.8ms
Speed: 4.7ms preprocess, 21.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 22.2ms
Speed: 3.6ms preprocess, 22.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 21.5ms
Speed: 4.8ms preprocess, 21.5ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 21.5ms
Speed: 3.2ms preprocess, 21.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.1ms
Speed: 3.1ms preprocess, 22.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.0ms
Speed: 3.3ms preprocess, 22.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)



In [ ]:
#存車子和車牌圖片到雲端 讚
import numpy as np
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import easyocr
import os

# 初始化 EasyOCR
reader = easyocr.Reader(['en'])

# 載入 YOLO 模型
model = YOLO('/content/drive/MyDrive/Finetune/Licence_Finetune/train/train/weights/best.pt')

# 輸入影片
cap = cv2.VideoCapture('/content/drive/MyDrive/Finetune/Licence_Finetune/test_video/IMG_0666.MOV')
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
videoWriter = cv2.VideoWriter("/content/drive/MyDrive/test-video1011.mp4", fourcc, fps, size)

# 初始化 DeepSort 追蹤器
tracker = DeepSort(max_age=60, max_cosine_distance=0.5, nn_budget=100)

# 定義顏色字典
colors = {
    'ambulance': (0, 255, 0),
    'bus': (0, 130, 255),
    'car': (0, 0, 255),
    'fire truck': (255, 0, 0),
    'licence': (60, 0, 128),
    'person': (255, 255, 0),
    'police': (0, 0, 0),
    'traffic light': (255, 0, 255)
}

# 確保所有截圖保存到雲端，車輛圖片資料夾為 'car_images'，車牌圖片資料夾為 'plate_images'
car_image_folder = '/content/drive/MyDrive/car_images'
plate_image_folder = '/content/drive/MyDrive/plate_images'

# 創建資料夾
if not os.path.exists(car_image_folder):
    os.makedirs(car_image_folder)
if not os.path.exists(plate_image_folder):
    os.makedirs(plate_image_folder)

# 用來存儲通過停止線的車輛資訊的列表
vehicles_passed_stop_line = []
recorded_vehicles = set()  # 記錄已經通過停止線並截圖的車輛ID

# 畫框標籤
def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))

    # 畫出矩形框
    cv2.rectangle(image, p1, p2, color, thickness=3, lineType=cv2.LINE_AA)
    if label:
        w, h = cv2.getTextSize(label, 0, fontScale=2 / 3, thickness=3)[0]
        outside = p1[1] - h >= 3
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)
        cv2.putText(image,
                    label, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                    0, 2/3, txt_color, thickness=3, lineType=cv2.LINE_AA)

# 車牌文字提取函數
def extract_license_plate_text(frame, bbox):
    # 截取車牌部分
    x1, y1, x2, y2 = list(map(int, bbox))
    license_plate_image = frame[y1:y2, x1:x2]

    # 使用 EasyOCR 來讀取車牌號碼
    result = reader.readtext(license_plate_image)
    if result:
        return result[0][-2]  # 提取最有可能的文字
    return ''

# 偵測車牌的過程
def detect_license_plates(outputs, frame):
    license_plates = []
    for output in outputs:
        class_id = int(output[5])
        if class_id == 4:  # '4' 是車牌的類別（假設）
            bbox = output[:4]
            license_plate_text = extract_license_plate_text(frame, bbox)  # 提取車牌文字
            if license_plate_text:  # 如果偵測到車牌文字，才加入列表
                license_plates.append({
                    'bbox': bbox,
                    'text': license_plate_text
                })
    return license_plates

# 保存車輛裁切圖片的函數，並存到 'car_images' 資料夾
def save_cropped_vehicle_image(frame, bbox, track_id, timestamp, license_plate_text):
    # 截取車輛圖像
    x1, y1, x2, y2 = list(map(int, bbox))
    cropped_vehicle = frame[y1:y2, x1:x2]

    # 生成文件名稱，包含秒數（後兩位）和車牌號碼
    timestamp_str = f"{int(timestamp):02d}"  # 秒數
    file_name = f"vehicle_{track_id}_time_{timestamp_str}_{license_plate_text}.jpg"
    file_path = os.path.join(car_image_folder, file_name)

    # 保存裁切後的圖片到 'car_images' 資料夾
    cv2.imwrite(file_path, cropped_vehicle)
    print(f"Vehicle image saved: {file_path}")

# 保存車牌截圖的函數，並存到 'plate_images' 資料夾
def save_license_plate_image(frame, bbox, license_plate_text):
    # 截取車牌圖像
    x1, y1, x2, y2 = list(map(int, bbox))
    cropped_plate = frame[y1:y2, x1:x2]

    # 生成文件名稱
    file_name = f"plate_{license_plate_text}.jpg"
    file_path = os.path.join(plate_image_folder, file_name)

    # 保存裁切後的車牌圖片到 'plate_images' 資料夾
    cv2.imwrite(file_path, cropped_plate)
    print(f"License plate image saved: {file_path}")

# 判斷車輛是否超過停止線，並且偵測到車牌號碼
def check_vehicle_passing_stop_line(tracks, frame_num, frame, license_plates):
    # 固定的停止線座標：x從630到900，y為430
    stop_line_y = 430
    for track in tracks:
        if track.det_class in ['car', 'bus', 'ambulance', 'fire truck']:  # 只考慮車輛類別
            bbox = track.to_ltrb()

            # 檢查車輛是否超過停止線且未被記錄
            if bbox[1] > stop_line_y and track.track_id not in recorded_vehicles:
                timestamp = frame_num / fps

                # 找到與車輛位置重疊的車牌
                license_plate_text = ''
                for license_plate in license_plates:
                    lp_bbox = license_plate['bbox']
                    if (lp_bbox[0] >= bbox[0] and lp_bbox[1] >= bbox[1] and
                        lp_bbox[2] <= bbox[2] and lp_bbox[3] <= bbox[3]):
                        license_plate_text = license_plate['text']  # 提取車牌號碼

                        # 保存車牌截圖
                        save_license_plate_image(frame, lp_bbox, license_plate_text)
                        break

                # 只有當偵測到車牌文字時才截圖並記錄
                if license_plate_text:
                    # 保存車輛ID、座標、車牌號碼、時間戳
                    vehicles_passed_stop_line.append({
                        'track_id': track.track_id,
                        'coordinates': bbox,
                        'license_plate': license_plate_text,
                        'timestamp': timestamp
                    })

                    # 記錄該車輛ID以避免重複截圖
                    recorded_vehicles.add(track.track_id)

                    # 畫框標籤
                    box_label(frame, bbox, f'Passed #{track.track_id} {license_plate_text}', (0, 255, 255))

                    # 保存裁切的車輛圖像
                    save_cropped_vehicle_image(frame, bbox, track.track_id, timestamp, license_plate_text)

# 主程式迴圈
frame_num = 0
while cap.isOpened():
    success, frame = cap.read()

    if success:
        frame_num += 1
        results = model(frame, conf=0.4, iou=0.8)  # 偵測物體
        outputs = results[0].boxes.data.cpu().numpy()

        detections = []

        if outputs is not None:
            # 偵測車牌
            license_plates = detect_license_plates(outputs, frame)

            for output in outputs:
                x1, y1, x2, y2 = list(map(int, output[:4]))
                class_id = int(output[5])

                label = ['ambulance', 'bus', 'car', 'fire truck', 'licence', 'person', 'police', 'traffic light'][class_id]
                color = colors.get(label, (128, 128, 128))
                detections.append(([x1, y1, int(x2 - x1), int(y2 - y1)], output[4], label))

            tracks = tracker.update_tracks(detections, frame=frame)  # 更新追蹤資訊

            check_vehicle_passing_stop_line(tracks, frame_num, frame, license_plates)  # 檢查車輛是否通過停止線

            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                bbox = track.to_ltrb()

                label = track.det_class
                color = colors.get(label, (128, 128, 128))
                box_label(frame, bbox, '#' + str(int(track_id)) + label, color)

        # 繪製固定的黃色停止線（x: 800-1500，y: 780-820）
        cv2.line(frame, (800, 820), (1500, 780), (0, 255, 255), 5)  # 黃色停止線

        cv2.putText(frame, "https://blog.csdn.net/zhaocj", (25, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        videoWriter.write(frame)

        if frame_num % 30 == 0:
            print(f"Processed frame {frame_num}")
    else:
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()


串流輸出內容已截斷至最後 5000 行。
Speed: 6.4ms preprocess, 24.1ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 23.4ms
Speed: 3.2ms preprocess, 23.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.5ms
Speed: 3.7ms preprocess, 22.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.5ms
Speed: 3.6ms preprocess, 22.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.5ms
Speed: 3.4ms preprocess, 22.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 22.5ms
Speed: 7.7ms preprocess, 22.5ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 25.7ms
Speed: 3.4ms preprocess, 25.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 22.5ms
Speed: 3.6ms preprocess, 22.5ms inference, 1.7ms postprocess per image at shape (1, 3, 3

In [ ]:
#整合程式

import mysql.connector
import pandas as pd
from datetime import datetime
from google.colab.patches import cv2_imshow
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pytesseract, cv2, PIL, glob, os
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# 載入 YOLO 模型
model = YOLO('/content/drive/MyDrive/Finetune/Licence_Finetune/train/train/weights/best.pt')

# 輸入影片
cap = cv2.VideoCapture('/content/drive/MyDrive/Finetune/Licence_Finetune/test_video/IMG_0666.MOV')
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
videoWriter = cv2.VideoWriter("/content/drive/MyDrive/test-video1014.mp4", fourcc, fps, size)

# 初始化 DeepSort 追蹤器
tracker = DeepSort(max_age=60, max_cosine_distance=0.5, nn_budget=100)

# 定義顏色字典
colors = {
    'ambulance': (0, 255, 0),
    'bus': (0, 130, 255),
    'car': (0, 0, 255),
    'fire truck': (255, 0, 0),
    'licence': (60, 0, 128),
    'person': (255, 255, 0),
    'police': (0, 0, 0),
    'traffic light': (255, 0, 255)
}

# 確保所有截圖保存到雲端，車輛圖片資料夾為 'car_images'，車牌圖片資料夾為 'plate_images'
car_image_folder = '/content/drive/MyDrive/car_images'
plate_image_folder = '/content/drive/MyDrive/plate_images'

# 創建資料夾
if not os.path.exists(car_image_folder):
    os.makedirs(car_image_folder)
if not os.path.exists(plate_image_folder):
    os.makedirs(plate_image_folder)

# 用來存儲通過停止線的車輛資訊的列表
vehicles_passed_stop_line = []
recorded_vehicles = set()  # 記錄已經通過停止線並截圖的車輛ID

# 畫框標籤
def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))

    # 畫出矩形框
    cv2.rectangle(image, p1, p2, color, thickness=3, lineType=cv2.LINE_AA)
    if label:
        w, h = cv2.getTextSize(label, 0, fontScale=2 / 3, thickness=3)[0]
        outside = p1[1] - h >= 3
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)
        cv2.putText(image,
                    label, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                    0, 2/3, txt_color, thickness=3, lineType=cv2.LINE_AA)

# 車牌文字提取函數
def extract_license_plate_text(frame, bbox):
    # 截取車牌部分
    x1, y1, x2, y2 = list(map(int, bbox))
    license_plate_image = frame[y1:y2, x1:x2]

    # 使用 EasyOCR 來讀取車牌號碼
    result = reader.readtext(license_plate_image)
    if result:
        return result[0][-2]  # 提取最有可能的文字
    return ''

# 偵測車牌的過程
def detect_license_plates(outputs, frame):
    license_plates = []
    for output in outputs:
        class_id = int(output[5])
        if class_id == 4:  # '4' 是車牌的類別（假設）
            bbox = output[:4]
            license_plate_text = extract_license_plate_text(frame, bbox)  # 提取車牌文字
            if license_plate_text:  # 如果偵測到車牌文字，才加入列表
                license_plates.append({
                    'bbox': bbox,
                    'text': license_plate_text
                })
    return license_plates

# 保存車輛裁切圖片的函數，並存到 'car_images' 資料夾
def save_cropped_vehicle_image(frame, bbox, track_id, timestamp, license_plate_text):
    # 截取車輛圖像
    x1, y1, x2, y2 = list(map(int, bbox))
    cropped_vehicle = frame[y1:y2, x1:x2]

    # 生成文件名稱，包含秒數（後兩位）和車牌號碼
    timestamp_str = f"{int(timestamp):02d}"  # 秒數
    file_name = f"vehicle_{track_id}_time_{timestamp_str}_{license_plate_text}.jpg"
    file_path = os.path.join(car_image_folder, file_name)

    # 保存裁切後的圖片到 'car_images' 資料夾
    cv2.imwrite(file_path, cropped_vehicle)
    print(f"Vehicle image saved: {file_path}")

# 保存車牌截圖的函數，並存到 'plate_images' 資料夾
def save_license_plate_image(frame, bbox, license_plate_text):
    # 截取車牌圖像
    x1, y1, x2, y2 = list(map(int, bbox))
    cropped_plate = frame[y1:y2, x1:x2]

    # 生成文件名稱
    file_name = f"plate_{license_plate_text}.jpg"
    file_path = os.path.join(plate_image_folder, file_name)

    # 保存裁切後的車牌圖片到 'plate_images' 資料夾
    cv2.imwrite(file_path, cropped_plate)
    print(f"License plate image saved: {file_path}")

# 判斷車輛是否超過停止線，並且偵測到車牌號碼
def check_vehicle_passing_stop_line(tracks, frame_num, frame, license_plates):
    # 固定的停止線座標：x從630到900，y為430
    stop_line_y = 430
    for track in tracks:
        if track.det_class in ['car', 'bus', 'ambulance', 'fire truck']:  # 只考慮車輛類別
            bbox = track.to_ltrb()

            # 檢查車輛是否超過停止線且未被記錄
            if bbox[1] > stop_line_y and track.track_id not in recorded_vehicles:
                timestamp = frame_num / fps

                # 找到與車輛位置重疊的車牌
                license_plate_text = ''
                for license_plate in license_plates:
                    lp_bbox = license_plate['bbox']
                    if (lp_bbox[0] >= bbox[0] and lp_bbox[1] >= bbox[1] and
                        lp_bbox[2] <= bbox[2] and lp_bbox[3] <= bbox[3]):
                        license_plate_text = license_plate['text']  # 提取車牌號碼

                        # 保存車牌截圖
                        save_license_plate_image(frame, lp_bbox, license_plate_text)
                        files=glob.glob(frame)
                        break

                # 只有當偵測到車牌文字時才截圖並記錄
                if license_plate_text:
                    # 保存車輛ID、座標、車牌號碼、時間戳
                    vehicles_passed_stop_line.append({
                        'track_id': track.track_id,
                        'coordinates': bbox,
                        'license_plate': license_plate_text,
                        'timestamp': timestamp
                    })

                    # 記錄該車輛ID以避免重複截圖
                    recorded_vehicles.add(track.track_id)

                    # 畫框標籤
                    box_label(frame, bbox, f'Passed #{track.track_id} {license_plate_text}', (0, 255, 255))

                    # 保存裁切的車輛圖像
                    save_cropped_vehicle_image(frame, bbox, track.track_id, timestamp, license_plate_text)
                    car_image_path = frame
                    plate_image_path = files
                    license_plate_text = text
                    cnx = connect_to_database()
                    cursor = cnx.cursor()
                    insert_vehicle_data(cnx, cursor, license_plate_text, datetime.now(), car_image_path, plate_image_path)
                    cursor.close()
                    cnx.close()

def photo_resize(img,target_width,target_height):
  # 調整圖片大小
  re_img=cv2.resize(img, (target_width, target_height))
  return re_img
#調整亮度與對比度
def adjusted(img,x,y):
  alpha = x
  beta = y

  adjusted_image = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

  return adjusted_image

def Sobel(img):
  Sobelx=cv2.Sobel(img,cv2.CV_32F,1,0)
  Sobelx=cv2.convertScaleAbs(Sobelx)
  return Sobelx

def Gussian(img):
  kernel_size = (5, 5)  # 高斯和大小 通常奇數
  sigma_x = 0  # X方向上的標準差，0 表示根據核大小自動計算
  # 高斯濾波器
  blurred_img = cv2.GaussianBlur(img, kernel_size, sigma_x)
  return blurred_img
def threshold(img):
  thr, binary = cv2.threshold(img,127, 255, cv2.THRESH_BINARY)
  return binary
def contours(img):
  contours1 = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #尋找輪廓
  contours = contours1[0]   #取得輪廓
  # cv2.drawContours(img, contours, -1, (0, 255, 0), 2)
  letters=[]
  for contour in contours:
    x,y,w,h=cv2.boundingRect(contour)
    if 15<h<85: #要改的參數!!!10<h<100
      letters.append(contour)
  # cv2.drawContours(contours_img, letters, -1, (255, 0, 0), 2)
  # cv2_imshow(contours_img)
  #找出最小x跟最大x
  min_x =float('inf')#正無窮
  max_x =0
  for letter in letters:
    for point in letter:
      x=point[0][0]
      min_x=min(min_x,x)
      max_x=max(max_x,x)
  return min_x,max_x
#image[y:h,x:w]
def cropped(img,x,y,h,w):
  cropped_img=img[y:h,x:w]

  return cropped_img

def erope(img):
  kernel=np.ones((3, 3),np.uint8)
  ero=cv2.erode(img,kernel,iterations=1)
  return ero

def dilate(img):
  kernel=np.ones((2, 5),np.uint8)
  dilation=cv2.dilate(img,kernel,iterations=8)
  return dilation

def findcontour(img):
  contours1 = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #尋找輪廓
  contours = contours1[0]   #取得輪廓
  points=None
  min_contour_length = 2500

  # 遍歷所有輪廓
  for contour in contours:
      # 計算輪廓的面積
      area = cv2.contourArea(contour)

      # 如果輪廓的面積大於2500，繪製最小外接矩形
      if area >= min_contour_length:
          # 計算輪廓的最小外接矩形
          rect = cv2.minAreaRect(contour)
          angle=rect[2]
          points = cv2.boxPoints(rect)
          points = np.int0(points)
          break

  if points is not None:
    if angle>=0:
      sorted_points = sorted(points, key=lambda x: x[0])
      bottom_left=sorted_points[0]
      top_left=sorted_points[1]
      bottom_right=sorted_points[2]
      top_right=sorted_points[3]

      return bottom_left,top_left,bottom_right,top_right
    else:
      sorted_points = sorted(points, key=lambda x: x[0])
      top_left=sorted_points[0]
      bottom_left=sorted_points[1]
      top_right=sorted_points[2]
      bottom_right=sorted_points[3]

      return bottom_left,top_left,bottom_right,top_right
  else:
    return 0,0,0,0

def affine(img,bottom_left,top_left,bottom_right,top_right):
  p1 = np.float32([(top_left),(top_right),(bottom_right)])
  p2 = np.float32([[top_left[0],top_right[1]],(top_right),(bottom_right)])
  M = cv2.getAffineTransform(p1, p2)

  h,w=adj_img.shape[:2]

  output = cv2.warpAffine(adj_img, M,(w,h))

  x=0
  y=min(top_right[1],bottom_right[1])
  h=abs(bottom_right[1]-top_right[1])

  output=output[y-5:y+h,x:x+w]
  # output=adjusted(output,-1.5,15)
  # output=adjusted(output,-1.3,12)
  # output=adjusted(output,-1.5,15)
  output=adjusted(output,-1.2,13)
  if output is not None:
    # output = cv2.cvtColor(output,cv2.COLOR_BGR2GRAY)
    # thr, output = cv2.threshold(output,127, 255, cv2.THRESH_BINARY)
    cv2.imwrite('output.jpg',output)
  else:
    print('None')
    pass

def bg(img):
  image=cv2.imread(img)
  height,width=image.shape[:2]
  new_height=height+36
  new_width=width+36
  x_offset=18
  y_offset=18
  #創建一個白色背景圖像
  background = np.ones((new_height, new_width, 3), dtype=np.uint8) * 255
  background[y_offset:y_offset+height,x_offset:x_offset+width]=image
  cv2.imwrite('output.jpg',background)

def tesseracttt(img):

  config = r'-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789. --oem 3 --psm 6'
  text = pytesseract.image_to_string(img,lang='eng',config=config)

  if len(text) ==1:
    print('辨識失敗')
  else:
    clean_text = text.strip()#去除兩端不可見字符
    print('辨識結果:\n{}'.format(clean_text))
    img=cv2.imread(img)
    # cv2.putText(img, clean_text, (int(110), int(70)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
    cv2_imshow(img)
    return text

# 連結到資料庫
def connect_to_database():
    config = {
        'user': '113403',
        'password': '@All3403@',
        'host': '140.131.114.242',
        'database': '113-law'
    }
    return mysql.connector.connect(**config)

# 讀取圖片並將其轉換為二進制 BLOB
def image_to_blob(image_path):
    with open(image_path, 'rb') as image_file:
        return image_file.read()

# 插入資料的函數，包含車輛和車牌圖片
def insert_vehicle_data(cnx, cursor, license_plate_text, timestamp, car_image_path, plate_image_path):
    try:
        # 讀取圖像 BLOB
        car_image_blob = image_to_blob(car_image_path)
        plate_image_blob = image_to_blob(plate_image_path)

        # 獲取當前時間
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # SQL插入语句，包含 BLOB 欄位
        add_violation = ("INSERT INTO testapp_car "
                         "(license_plate, date_time, location, violation, seconds, car_image, plate_image) "
                         "VALUES (%s, %s, %s, %s, %s, %s, %s)")

        # 插入資料
        data_violation = (
            license_plate_text,
            current_time,
            'TAIPEI',  # 假設地點
            '闖紅燈',  # 假設違規類型
            5.00,       # 假設秒數
            car_image_blob,  # 車輛圖片 BLOB
            plate_image_blob  # 車牌圖片 BLOB
        )

        cursor.execute(add_violation, data_violation)
        cnx.commit()  # 使用連接物件提交事務
        print("資料插入成功")

    except mysql.connector.Error as err:
        print(f"插入資料時發生錯誤: {err}")
    except Exception as e:
        print(f"其他錯誤: {e}")

# 主程式迴圈
frame_num = 0
while cap.isOpened():
    success, frame = cap.read()

    if success:
        frame_num += 1
        results = model(frame, conf=0.4, iou=0.8)  # 偵測物體
        outputs = results[0].boxes.data.cpu().numpy()

        detections = []

        if outputs is not None:
            # 偵測車牌
            license_plates = detect_license_plates(outputs, frame)

            for output in outputs:
                x1, y1, x2, y2 = list(map(int, output[:4]))
                class_id = int(output[5])

                label = ['ambulance', 'bus', 'car', 'fire truck', 'licence', 'person', 'police', 'traffic light'][class_id]
                color = colors.get(label, (128, 128, 128))
                detections.append(([x1, y1, int(x2 - x1), int(y2 - y1)], output[4], label))

            tracks = tracker.update_tracks(detections, frame=frame)  # 更新追蹤資訊

            check_vehicle_passing_stop_line(tracks, frame_num, frame, license_plates)  # 檢查車輛是否通過停止線

            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                bbox = track.to_ltrb()

                label = track.det_class
                color = colors.get(label, (128, 128, 128))
                box_label(frame, bbox, '#' + str(int(track_id)) + label, color)

        # 繪製固定的黃色停止線（x: 800-1500，y: 780-820）
        cv2.line(frame, (800, 820), (1500, 780), (0, 255, 255), 5)  # 黃色停止線

        cv2.putText(frame, "https://blog.csdn.net/zhaocj", (25, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        videoWriter.write(frame)

        if frame_num % 30 == 0:
            print(f"Processed frame {frame_num}")
    else:
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()



0: 384x640 1 licence, 40.8ms
Speed: 2.6ms preprocess, 40.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'imread'
> Overload resolution failed:
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object


In [ ]:
#chat測試 車牌不夠準
import cv2
import numpy as np
import pytesseract
from google.colab.patches import cv2_imshow
import mysql.connector
from datetime import datetime
import os
import easyocr
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# 初始化 EasyOCR
reader = easyocr.Reader(['en'])

# 載入 YOLO 模型
model = YOLO('/content/drive/MyDrive/Finetune/Licence_Finetune/train/train/weights/best.pt')

# 輸入影片
cap = cv2.VideoCapture('/content/drive/MyDrive/Finetune/Licence_Finetune/test_video/IMG_0666.MOV')
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
videoWriter = cv2.VideoWriter("/content/drive/MyDrive/test-video1014.mp4", fourcc, fps, size)

# 初始化 DeepSort 追蹤器
tracker = DeepSort(max_age=60, max_cosine_distance=0.5, nn_budget=100)

# 定義顏色字典
colors = {
    'ambulance': (0, 255, 0),
    'bus': (0, 130, 255),
    'car': (0, 0, 255),
    'fire truck': (255, 0, 0),
    'licence': (60, 0, 128),
    'person': (255, 255, 0),
    'police': (0, 0, 0),
    'traffic light': (255, 0, 255)
}

# 確保所有截圖保存到雲端，車輛圖片資料夾為 'car_images'，車牌圖片資料夾為 'plate_images'
car_image_folder = '/content/drive/MyDrive/car_images'
plate_image_folder = '/content/drive/MyDrive/plate_images'

# 創建資料夾
os.makedirs(car_image_folder, exist_ok=True)
os.makedirs(plate_image_folder, exist_ok=True)

# 用來存儲通過停止線的車輛資訊的列表
vehicles_passed_stop_line = []
recorded_vehicles = set()  # 記錄已經通過停止線並截圖的車輛ID

# 畫框標籤
def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
    cv2.rectangle(image, p1, p2, color, thickness=3, lineType=cv2.LINE_AA)
    if label:
        w, h = cv2.getTextSize(label, 0, fontScale=2 / 3, thickness=3)[0]
        outside = p1[1] - h >= 3
        p2_text = (p1[0] + w, p1[1] - h - 3) if outside else (p1[0] + w, p1[1] + h + 3)
        cv2.rectangle(image, p1, p2_text, color, -1, cv2.LINE_AA)
        cv2.putText(image,
                    label, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                    0, 2/3, txt_color, thickness=3, lineType=cv2.LINE_AA)

# 車牌文字提取函數
def extract_license_plate_text(frame, bbox):
    # 截取車牌部分
    x1, y1, x2, y2 = list(map(int, bbox))
    license_plate_image = frame[y1:y2, x1:x2]
    result = reader.readtext(license_plate_image)
    if result:
        return result[0][-2]  # 提取最有可能的文字
    return ''

# 偵測車牌的過程
def detect_license_plates(outputs, frame):
    license_plates = []
    for output in outputs:
        class_id = int(output[5])
        if class_id == 4:  # '4' 是車牌的類別（假設）
            bbox = output[:4]
            license_plate_text = extract_license_plate_text(frame, bbox)  # 提取車牌文字
            if license_plate_text:  # 如果偵測到車牌文字，才加入列表
                license_plates.append({
                    'bbox': bbox,
                    'text': license_plate_text
                })
    return license_plates

# 保存車輛裁切圖片的函數，並存到 'car_images' 資料夾
def save_cropped_vehicle_image(frame, bbox, track_id, timestamp, license_plate_text):
    # 截取車輛圖像
    x1, y1, x2, y2 = list(map(int, bbox))
    cropped_vehicle = frame[y1:y2, x1:x2]
    # 生成文件名稱，包含秒數（後兩位）和車牌號碼
    timestamp_str = f"{int(timestamp):02d}"  # 秒數
    file_name = f"vehicle_{track_id}_time_{timestamp_str}_{license_plate_text}.jpg"
    file_path = os.path.join(car_image_folder, file_name)
    # 保存裁切後的圖片到 'car_images' 資料夾
    cv2.imwrite(file_path, cropped_vehicle)
    print(f"Vehicle image saved: {file_path}")
    return file_path  # 返回路徑以便後續使用

# 保存車牌截圖的函數，並存到 'plate_images' 資料夾
def save_license_plate_image(frame, bbox, license_plate_text):
    # 截取車牌圖像
    x1, y1, x2, y2 = list(map(int, bbox))
    cropped_plate = frame[y1:y2, x1:x2]
    # 生成文件名稱
    file_name = f"plate_{license_plate_text}.jpg"
    file_path = os.path.join(plate_image_folder, file_name)
    # 保存裁切後的車牌圖片到 'plate_images' 資料夾
    cv2.imwrite(file_path, cropped_plate)
    print(f"License plate image saved: {file_path}")
    return file_path  # 返回路徑以便後續使用

# 判斷車輛是否超過停止線，並且偵測到車牌號碼
def check_vehicle_passing_stop_line(tracks, frame_num, frame, license_plates):
    # 固定的停止線座標：x從630到900，y為430
    stop_line_y = 430
    for track in tracks:
        if track.det_class in ['car', 'bus', 'ambulance', 'fire truck']:  # 只考慮車輛類別
            bbox = track.to_ltrb()
            # 檢查車輛是否超過停止線且未被記錄
            if bbox[1] > stop_line_y and track.track_id not in recorded_vehicles:
                timestamp = frame_num / fps
                license_plate_text = ''
                plate_image_path = ''
                car_image_path = ''
                for license_plate in license_plates:
                    lp_bbox = license_plate['bbox']
                    if (lp_bbox[0] >= bbox[0] and lp_bbox[1] >= bbox[1] and
                        lp_bbox[2] <= bbox[2] and lp_bbox[3] <= bbox[3]):
                        license_plate_text = license_plate['text']
                        # 保存車牌截圖並獲取路徑
                        plate_image_path = save_license_plate_image(frame, lp_bbox, license_plate_text)
                        break
                if license_plate_text:
                    # 保存車輛裁切圖片並獲取路徑
                    car_image_path = save_cropped_vehicle_image(frame, bbox, track.track_id, timestamp, license_plate_text)
                    # 畫框標籤
                    box_label(frame, bbox, f'Passed #{track.track_id} {license_plate_text}', (0, 255, 255))
                    # 保存車輛資訊
                    vehicles_passed_stop_line.append({
                        'track_id': track.track_id,
                        'coordinates': bbox,
                        'license_plate': license_plate_text,
                        'timestamp': timestamp
                    })
                    # 記錄該車輛ID以避免重複截圖
                    recorded_vehicles.add(track.track_id)
                    # 連結資料庫並插入資料
                    cnx = connect_to_database()
                    cursor = cnx.cursor()
                    insert_vehicle_data(cnx, cursor, license_plate_text, timestamp, car_image_path, plate_image_path)
                    cursor.close()
                    cnx.close()

# 連結到資料庫
def connect_to_database():
    config = {
        'user': '113403',
        'password': '@All3403@',
        'host': '140.131.114.242',
        'database': '113-law'
    }
    return mysql.connector.connect(**config)

# 讀取圖片並將其轉換為二進制 BLOB
def image_to_blob(image_path):
    with open(image_path, 'rb') as image_file:
        return image_file.read()

# 插入資料的函數，包含車輛和車牌圖片
def insert_vehicle_data(cnx, cursor, license_plate_text, timestamp, car_image_path, plate_image_path):
    try:
        # 讀取圖像 BLOB
        car_image_blob = image_to_blob(car_image_path)
        plate_image_blob = image_to_blob(plate_image_path)
        # 獲取當前時間
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        # SQL插入语句，包含 BLOB 欄位
        add_violation = ("INSERT INTO testapp_car "
                         "(license_plate, date_time, location, violation, seconds, car_image, plate_image) "
                         "VALUES (%s, %s, %s, %s, %s, %s, %s)")
        # 插入資料
        data_violation = (
            license_plate_text,
            current_time,
            'TAIPEI',  # 假設地點
            '闖紅燈',  # 假設違規類型
            round(timestamp - 0.5, 2),  # 偵測到車子當下的秒數再往前0.5秒
            car_image_blob,  # 車輛圖片 BLOB
            plate_image_blob  # 車牌圖片 BLOB
        )
        cursor.execute(add_violation, data_violation)
        cnx.commit()  # 使用連接物件提交事務
        print("資料插入成功")
    except mysql.connector.Error as err:
        print(f"插入資料時發生錯誤: {err}")
    except Exception as e:
        print(f"其他錯誤: {e}")

# 主程式迴圈
frame_num = 0
while cap.isOpened():
    success, frame = cap.read()
    if success:
        frame_num += 1
        results = model(frame, conf=0.4, iou=0.8)  # 偵測物體
        outputs = results[0].boxes.data.cpu().numpy()

        detections = []

        if outputs is not None:
            # 偵測車牌
            license_plates = detect_license_plates(outputs, frame)

            for output in outputs:
                x1, y1, x2, y2, conf, class_id = output
                class_id = int(class_id)
                if class_id >= len(colors):
                    label = 'unknown'
                else:
                    label = list(colors.keys())[class_id]
                color = colors.get(label, (128, 128, 128))
                detections.append(([x1, y1, int(x2 - x1), int(y2 - y1)], conf, label))

            tracks = tracker.update_tracks(detections, frame=frame)  # 更新追蹤資訊

            check_vehicle_passing_stop_line(tracks, frame_num, frame, license_plates)  # 檢查車輛是否通過停止線

            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                bbox = track.to_ltrb()
                label = track.det_class
                color = colors.get(label, (128, 128, 128))
                box_label(frame, bbox, f'#{track_id} {label}', color)

        # 繪製固定的黃色停止線（x: 800-1500，y: 430）
        cv2.line(frame, (630, 430), (900, 430), (0, 255, 255), 5)  # 黃色停止線

        cv2.putText(frame, "https://blog.csdn.net/zhaocj", (25, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        videoWriter.write(frame)

        if frame_num % 30 == 0:
            print(f"Processed frame {frame_num}")
    else:
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()


Progress: |██████████████████████████████████████████████████| 100.0% Complete

串流輸出內容已截斷至最後 5000 行。
Speed: 3.0ms preprocess, 28.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 29.4ms
Speed: 9.3ms preprocess, 29.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 36.7ms
Speed: 5.1ms preprocess, 36.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 300

0: 384x640 1 car, 52.9ms
Speed: 10.6ms preprocess, 52.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 33.7ms
Speed: 8.2ms preprocess, 33.7ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 32.4ms
Speed: 2.8ms preprocess, 32.4ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 37.8ms
Speed: 5.9ms preprocess, 37.8ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 34.0ms
Speed: 6.7ms preprocess, 34.0ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)
